In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: kkamalesh. Use `wandb login --relogin` to force relogin


True

In [22]:
torch.__version__

'1.10.0'

In [ ]:
!conda uninstall pytorch

In [ ]:
#!pip install transformers[sentencepiece]
!pip install datasets
!pip install nltk
!pip install rouge_score
!pip install evaluate

In [3]:
wandb.init(project="T5-small testing", entity="kkamalesh")

In [4]:
from transformers import T5Tokenizer,T5ForConditionalGeneration,DataCollatorForSeq2Seq
from torch.utils.data import Dataset,DataLoader
from torch.nn.functional import relu
from evaluate import load
from datasets import load_metric
import pandas as pd
from collections import Counter
import numpy as np
import torch
from pprint import pprint
from torch import optim
from tqdm import tqdm
#torch.multiprocessing.set_start_method('spawn')
import os

BATCH_SIZE=1

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [4]:
!nvidia-smi

Tue Jun 28 20:28:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#df_plain=pd.read_csv('sample_text_2000samples_without_attributes.csv')
df_combined=pd.read_csv('sample_text_with_attributes_auglow.csv')[['Augmented text','Target Text']]
#df_attributes.rename(columns={'Augmented text':'Augmented Text'},inplace=True)
#df_combined=pd.concat([df_plain[['Augmented Text','Target Text']],df_attributes[['Augmented Text','Target Text']]])
df_combined=df_combined.sample(frac=1)
df=df_combined.applymap(lambda x:x.replace('\n',' [LINE] '))

In [7]:
df_train=df.iloc[:1600,:]
df_test=df.iloc[1600:,:]
len_train=df_train.shape[0]
len_test=df_test.shape[0]

In [8]:
tokenizer=T5Tokenizer.from_pretrained('t5-small',model_max_length=int(1e30))
model=T5ForConditionalGeneration.from_pretrained('t5-small')

In [8]:
!nvidia-smi

Tue Jun 28 20:28:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
print(df['Target Text'][1])

HARRIS TEETER, SHREDDED WHEAT CEREAL [LINE] WHOLE GRAIN WHEAT, SUGAR, SORBITOL, GELATIN, REDUCED IRON, NIACIN (NIACINAMIDE), ZINC (ZINC OXIDE), VITAMIN B6 (PYRIDOXINE HYDROCHLORIDE), FOLATE (FOLIC ACID), VITAMIN B2 (RIBOFLAVIN), VITAMIN B1 (THIAMIN HYDROCHLORIDE), VITAMIN B12, BHT (TO PRESERVE FRESHNESS). [LINE] Serving  1.0 cup [LINE] Quantity: 150g (Poids net égoutté : 90g) [LINE] Protein 9.09 g [LINE] Total lipid (fat) 1.82 g [LINE] Carbohydrate, by difference 81.82 g [LINE] Energy 345.0 kcal [LINE] Sugars, total 20.0 g [LINE] Carbohydrate, other 51.0 g [LINE] Fiber, total dietary 10.9 g [LINE] Fiber, soluble 2.0 g [LINE] Fiber, insoluble 9.0 g [LINE] Calcium, Ca 0.0 mg [LINE] Iron, Fe 29.45 mg [LINE] Magnesium, Mg 109.0 mg [LINE] Phosphorus, P 273.0 mg [LINE] Potassium, K 327.0 mg [LINE] Sodium, Na 18.0 mg [LINE] Zinc, Zn 6.82 mg [LINE] Copper, Cu 0.364 mg [LINE] Vitamin A, IU 0.0 IU [LINE] Vitamin C, total ascorbic acid 0.0 mg [LINE] Thiamin 1.0 mg [LINE] Riboflavin 0.773 mg [LINE

In [9]:
f=tokenizer(df['Target Text'][1])

In [9]:
print(tokenizer.decode(f['input_ids']))

HARRIS TEETER, SHREDDED WHEAT CEREAL [LINE] WHOLE GRAIN WHEAT, SUGAR, SORBITOL, GELATIN, REDUCED IRON, NIACIN (NIACINAMIDE), ZINC (ZINC OXIDE), VITAMIN B6 (PYRIDOXINE HYDROCHLORIDE), FOLATE (FOLIC ACID), VITAMIN B2 (RIBOFLAVIN), VITAMIN B1 (THIAMIN HYDROCHLORIDE), VITAMIN B12, BHT (TO PRESERVE FRESHNESS). [LINE] Serving 1.0 cup [LINE] Quantity: 150g (Poids net égoutté : 90g) [LINE] Protein 9.09 g [LINE] Total lipid (fat) 1.82 g [LINE] Carbohydrate, by difference 81.82 g [LINE] Energy 345.0 kcal [LINE] Sugars, total 20.0 g [LINE] Carbohydrate, other 51.0 g [LINE] Fiber, total dietary 10.9 g [LINE] Fiber, soluble 2.0 g [LINE] Fiber, insoluble 9.0 g [LINE] Calcium, Ca 0.0 mg [LINE] Iron, Fe 29.45 mg [LINE] Magnesium, Mg 109.0 mg [LINE] Phosphorus, P 273.0 mg [LINE] Potassium, K 327.0 mg [LINE] Sodium, Na 18.0 mg [LINE] Zinc, Zn 6.82 mg [LINE] Copper, Cu 0.364 mg [LINE] Vitamin A, IU 0.0 IU [LINE] Vitamin C, total ascorbic acid 0.0 mg [LINE] Thiamin 1.0 mg [LINE] Riboflavin 0.773 mg [LINE]

In [79]:
tk=tokenizer('My name is KAMALESH')

In [80]:
tokenizer.decode(tk.input_ids)

'My name is KAMALESH</s>'

In [9]:
class CreateDataset(Dataset):
    def __init__(self,df):
        source=df.iloc[:,0].values
        target=df.iloc[:,1].values
        self.source_token=tokenizer(list(source))
        self.target_token=tokenizer(list(target))
        self.len=len(source)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self,idx):
        #self.source_token.input_ids[idx]=torch.tensor(self.source_token.input_ids[idx],device=device,dtype=torch.float32)
        #self.source_token.attention_mask[idx]=torch.tensor(self.source_token.attention_mask[idx],device=device,dtype=torch.float32)
        #self.target_token.input_ids[idx]=torch.tensor(self.target_token.input_ids[idx],device=device,dtype=torch.float32)

        return {'input_ids':self.source_token.input_ids[idx],
               'attention_mask':self.source_token.attention_mask[idx],'labels': self.target_token.input_ids[idx]}

In [ ]:
df_test

In [10]:
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,return_tensors='pt')
train_set=CreateDataset(df_train)
test_set=CreateDataset(df_test)
train_loader=DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True,collate_fn=data_collator,pin_memory=True)
test_loader=DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True,collate_fn=data_collator,pin_memory=True)

In [14]:
for batch in train_loader:
    break

In [15]:
batch

{'input_ids': tensor([[ 4800, 16375, 24786,  ...,     0,     0,     0],
         [   71, 16976, 24203,  ...,     0,     0,     0],
         [    3,   122,   332,  ...,     0,     0,     0],
         ...,
         [    3, 10966, 11012,  ...,     0,     0,     0],
         [  377, 20291,   354,  ...,     0,     0,     0],
         [    3,  8579,  5249,  ..., 19120,   584,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([[ 4800, 16375, 24786,  ...,  -100,  -100,  -100],
         [   71, 16976, 24203,  ...,  -100,  -100,  -100],
         [    3, 25430, 13431,  ...,  -100,  -100,  -100],
         ...,
         [    3, 10966,   332,  ...,  -100,  -100,  -100],
         [  377, 20291,  6299,  ...,  -100,  -100,  -100],
         [ 2847, 13458,  5091,  ...,  -100,  -100,  -1

model.to(device)
batch_train=[]
batch_test=[]
for batch in test_loader:
    batch_test.append({key:value.to(device) for key,value in batch.items()})
for batch in train_loader:
    batch_train.append({key:value.to(device) for key,value in batch.items()})

In [22]:
model.device

device(type='cuda', index=0)

In [ ]:
model.to(device)

In [18]:
batch={key:value.to(device) for key,value in batch.items()}

In [12]:
bleu=load('bleu')
rouge=load('rouge')
train_lis=[]
test_lis=[]
def compute_metric(predicted,target,epoch,flag):
    predicted=[tokenizer.decode(x,skip_special_tokens=True) for x in predicted]
    target_bleu=[[tokenizer.decode(relu(x),skip_special_tokens=True)] for x in target]
    target_rouge=[tokenizer.decode(relu(x),skip_special_tokens=True) for x in target]
    bleu_score=bleu.compute(predictions=predicted,references=target_bleu)['bleu']
    rouge_score=rouge.compute(predictions=predicted,references=target_rouge,use_aggregator=False)['rouge1']
    precision=sum([x[0] for x in rouge_score])/BATCH_SIZE
    recall=sum([x[1] for x in rouge_score])/BATCH_SIZE
    fscore=sum([x[2] for x in rouge_score])/BATCH_SIZE
    #print(predicted[0].device,bleu_score.device)
    if flag==0:
        if epoch==num_epochs-1:
            train_lis.extend(predicted)
        return Counter({'bleu_train':bleu_score,'precision_train':precision,'recall_train':recall,'f-score_train':fscore})
    else:
        if epoch==num_epochs-1:
            test_lis.extend(predicted)
        return Counter({'bleu_test':bleu_score,'precision_test':precision,'recall_test':recall,'f-score_test':fscore})

In [14]:
!nvidia-smi

Tue Jun 28 09:13:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   40C    P0    34W /  70W |  11618MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%wandb
optimiser=optim.Adam(model.parameters())
num_epochs=10
#f=0
for epoch in tqdm(range(num_epochs)):
    metrics_train=Counter({'bleu_train':0,'precision_train':0,'recall_train':0,'f-score_train':0})
    metrics_test=Counter({'bleu_test':0,'precision_test':0,'recall_test':0,'f-score_test':0})
    model.train()
    for batch in train_loader:
        #print(f)
        batch={key:value.to(device) for key,value in batch.items()}
        #print('crossed-1')
        outputs=model(**batch)
        #print('crossed-2')
        loss=outputs.loss
        predicted_tokens=model.generate(batch['input_ids'])
        #print('crossed-3')
        metrics=compute_metric(predicted_tokens,batch['labels'],epoch=epoch,flag=0)
        #print('crossed-4')
        metrics_train+=metrics
        loss.backward()
        #print('crossed-5')
        optimiser.step()
        #print('crossed-6')
        optimiser.zero_grad()
        #print('crossed-7')
        #f+=1
        #print('crossed-8')
        torch.cuda.empty_cache()
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            batch={key:value.to(device) for key,value in batch.items()}
            predicted_tokens=model.generate(batch['input_ids'])
            metrics=compute_metric(predicted_tokens,batch['labels'],epoch=epoch,flag=1)
            metrics_test+=metrics
            torch.cuda.empty_cache()
    metrics_train={key:values*(BATCH_SIZE/len_train) for key,values in dict(metrics_train).items()}
    metrics_test={key:values*(BATCH_SIZE/len_test) for key,values in dict(metrics_test).items()}
    print('f-score_train: ',metrics_train['f-score_train'],' f-score_test: ',metrics_test['f-score_test'],' bleu_test: ',metrics_test['bleu_test'],' bleu_train: ',metrics_train['bleu_train'])
    wandb.log ({**metrics_train,**metrics_test})         

 10%|█         | 1/10 [21:36<3:14:30, 1296.70s/it]

f-score_train:  0.05024949360648866  f-score_test:  0.055081901292791856  bleu_test:  2.3917069034283688e-06  bleu_train:  3.4353317368101124e-06


 20%|██        | 2/10 [43:12<2:52:48, 1296.12s/it]

f-score_train:  0.0538752437037848  f-score_test:  0.05948982552827332  bleu_test:  1.117953770557857e-05  bleu_train:  3.902505394294708e-06


 30%|███       | 3/10 [1:04:51<2:31:23, 1297.58s/it]

...

f-score_train:  0.0644247845859147  f-score_test:  0.06196341538707536  bleu_test:  6.302855485599692e-06  bleu_train:  9.116386529411074e-06


 80%|████████  | 8/10 [2:53:16<43:22, 1301.11s/it]  

f-score_train:  0.0658932557312346  f-score_test:  0.06323602657063412  bleu_test:  4.482877789440081e-06  bleu_train:  1.2662239280085717e-05


 90%|█████████ | 9/10 [3:14:57<21:41, 1301.11s/it]

f-score_train:  0.06702018133160935  f-score_test:  0.06365804769472094  bleu_test:  4.860335538994294e-06  bleu_train:  1.696791813792668e-05


In [26]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   13663 MB |   13663 MB |   17420 MB |    3757 MB |
|       from large pool |   13662 MB |   13662 MB |   17412 MB |    3750 MB |
|       from small pool |       1 MB |       2 MB |       8 MB |       7 MB |
|---------------------------------------------------------------------------|
| Active memory         |   13663 MB |   13663 MB |   17420 MB |    3757 MB |
|       from large pool |   13662 MB |   13662 MB |   17412 MB |

In [ ]:
pprint({**metrics_train,**metrics_test})

In [ ]:
test_lis

In [ ]:
target_test=[]
for batch in test_loader:
    temp_batch=[tokenizer.decode(relu(x),skip_special_tokens=True) for x in batch['labels']]
    target_test.extend(temp_batch)

In [ ]:
final_df=pd.DataFrame(list(zip(test_lis,target_test)),columns=['Predicted','Target'])
final_df.to_csv('T5-small-results.csv')